In [139]:
from splinter import Browser
from bs4 import BeautifulSoup

In [140]:
item_name = input('What item are you looking for:' )
item_name = item_name.replace(" ", "+").lower()
item_name

'water+rune'

In [141]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [142]:
url = f'http://www.itemdb.biz/index.php?search={item_name}'
browser.visit(url)


In [143]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

table_row = soup.find_all('tbody')

for th in table_row:
    item_id = th.find('td').text
    item_name = th.find('td').next_element.next_element.next_element.next_element.next_element
    item_dict = {
        'item_name':item_name,
        'id': item_id
    }
    print(item_dict)

{'item_name': 'Water rune', 'id': '555'}
